In [1]:
rawTextRDD = sc.textFile("/FileStore/tables/shortStackOverflow.txt")
rawTextRDD.take(100)

Out[ 137 ]: ['1,100,102,,2,Python',
 '2,101,,100,5,',
 '2,102,,100,9,',
 '2,103,,100,3,',
 '1,104,,,2,Python',
 '2,105,,104,8,',
 '2,106,,104,5,',
 '1,200,,,2,C#',
 '2,201,,200,5,',
 '2,202,,200,1,',
 '2,203,,200,8,',
 '2,204,,200,8,',
 '2,205,,200,8,',
 '1,206,,,1,C#',
 '2,207,,206,11,',
 '2,208,,206,6,',
 '1,300,,,9,Java',
 '1,301,304,,7,Java',
 '2,302,,301,9,',
 '2,303,,301,3,',
 '2,304,,301,9,',
 '1,305,306,,3,Java',
 '2,306,,305,5,',
 '1,307,309,,3,Java',
 '2,308,,307,5,',
 '2,309,,307,5,',
 '1,400,402,,2,PHP',
 '2,401,,400,2,',
 '2,402,,400,7,',
 '1,403,404,,2,PHP',
 '2,404,,403,2,']

In [2]:
def isSolved(answerId):
  if len(answerId)>0:
    return True
  else:
    return False
  
def convert(line):
  fields =line.split(',')
  if int(fields[0]) == 1:
    return ( int(fields[1]), (1, 0, isSolved(fields[2]), fields[5])) 
  else:
    return ( int(fields[3]), (0,1) )                                 

questionsAndAnswersRDD = rawTextRDD.map(convert)
questionsAndAnswersRDD.take(100)

Out[ 138 ]: [(100, (1, 0, True, 'Python')),
 (100, (0, 1)),
 (100, (0, 1)),
 (100, (0, 1)),
 (104, (1, 0, False, 'Python')),
 (104, (0, 1)),
 (104, (0, 1)),
 (200, (1, 0, False, 'C#')),
 (200, (0, 1)),
 (200, (0, 1)),
 (200, (0, 1)),
 (200, (0, 1)),
 (200, (0, 1)),
 (206, (1, 0, False, 'C#')),
 (206, (0, 1)),
 (206, (0, 1)),
 (300, (1, 0, False, 'Java')),
 (301, (1, 0, True, 'Java')),
 (301, (0, 1)),
 (301, (0, 1)),
 (301, (0, 1)),
 (305, (1, 0, True, 'Java')),
 (305, (0, 1)),
 (307, (1, 0, True, 'Java')),
 (307, (0, 1)),
 (307, (0, 1)),
 (400, (1, 0, True, 'PHP')),
 (400, (0, 1)),
 (400, (0, 1)),
 (403, (1, 0, True, 'PHP')),
 (403, (0, 1))]

In [3]:
def sumReduction(a, b):
    if a[0] == 0 and b[0] == 0:
        return 0, a[1] + b[1]
    elif a[0] == 1:
        return 1, a[1] + b[1], a[2], a[3]
    else:
        return 1, a[1] + b[1], b[2], b[3]
  
joinedRDD = questionsAndAnswersRDD.reduceByKey(sumReduction)
joinedRDD.take(10)

Out[ 139 ]: [(400, (1, 2, True, 'PHP')),
 (200, (1, 5, False, 'C#')),
 (100, (1, 3, True, 'Python')),
 (104, (1, 2, False, 'Python')),
 (300, (1, 0, False, 'Java')),
 (206, (1, 2, False, 'C#')),
 (305, (1, 1, True, 'Java')),
 (403, (1, 1, True, 'PHP')),
 (301, (1, 3, True, 'Java')),
 (307, (1, 2, True, 'Java'))]

In [4]:
def function(pair):
    return ((pair[1][3], pair[1][2]), pair[1][1])


tagRDD = joinedRDD.map(function)

tagRDD.take(10)

Out[ 132 ]: [(('PHP', True), 2),
 (('C#', False), 5),
 (('Python', True), 3),
 (('Python', False), 2),
 (('Java', False), 0),
 (('C#', False), 2),
 (('Java', True), 1),
 (('PHP', True), 1),
 (('Java', True), 3),
 (('Java', True), 2)]

In [5]:
reducedTagRDD = tagRDD.combineByKey(
    (lambda x: (x, 1)),
    (lambda x, y: (x[0] + y, x[1] + 1)),
    (lambda x, y: (x[0] + y[0], x[1] + y[1]))
)
reducedTagRDD.take(10)

Out[ 140 ]: [(('Java', False), (0, 1)),
 (('Python', False), (2, 1)),
 (('C#', False), (7, 2)),
 (('Java', True), (6, 3)),
 (('Python', True), (3, 1)),
 (('PHP', True), (3, 2))]

In [6]:
def extraction(pair):
    key = pair[0]
    value = pair[1]
    tag = key[0]
    isSolved = key[1]
    numberOfQuestions = value[1]
    n_times = value[0]
    if not isSolved:
        return tag, (0, 'nan', numberOfQuestions, n_times / numberOfQuestions)
    else:
        return tag, (numberOfQuestions, n_times / numberOfQuestions, 0, 'nan')

statisticsRDD = reducedTagRDD.map(extraction)
statisticsRDD.take(10)

Out[ 141 ]: [('Java', (0, 'nan', 1, 0.0)),
 ('Python', (0, 'nan', 1, 2.0)),
 ('C#', (0, 'nan', 2, 3.5)),
 ('Java', (3, 2.0, 0, 'nan')),
 ('Python', (1, 3.0, 0, 'nan')),
 ('PHP', (2, 1.5, 0, 'nan'))]

In [7]:
def reduccion(a, b):
    if a[1] == 'nan':
        return b[0], b[1], a[2], a[3]
    else:
        return a[0], a[1], b[2], b[3]

RBK_RDD = statisticsRDD.reduceByKey(reduccion)
RBK_RDD.take(10)

Out[ 142 ]: [('Java', (3, 2.0, 1, 0.0)),
 ('PHP', (2, 1.5, 0, 'nan')),
 ('Python', (1, 3.0, 1, 2.0)),
 ('C#', (0, 'nan', 2, 3.5))]

In [8]:
RBK_RDD.sortBy(lambda a: a[1])
RBK_RDD.take(10)

Out[ 148 ]: [('Java', (3, 2.0, 1, 0.0)),
 ('PHP', (2, 1.5, 0, 'nan')),
 ('Python', (1, 3.0, 1, 2.0)),
 ('C#', (0, 'nan', 2, 3.5))]

In [9]:
rawTextRDD = sc.textFile("/FileStore/tables/shortStackOverflow.txt")

def isSolved(answerId):
    if len(answerId) > 0:
        return True
    else:
        return False


def convert(line):
    fields = line.split(',')
    if int(fields[0]) == 1:
        return int(fields[1]), (1, 0, isSolved(fields[2]), fields[5])
    else:
        return int(fields[3]), (0, 1)

questionsAndAnswersRDD = rawTextRDD.map(convert)

def sumaVeces(a, b):
    if a[0] == 0 and b[0] == 0:
        return 0, a[1] + b[1]
    elif a[0] == 1:
        return 1, a[1] + b[1], a[2], a[3]
    else:
        return 1, a[1] + b[1], b[2], b[3]
  
sumadoRDD = questionsAndAnswersRDD.reduceByKey(sumaVeces)


def function(pair):
    return (pair[1][3], pair[1][2]), pair[1][1]

tagRDD = sumadoRDD.map(function)


reducidoTagRDD = tagRDD.combineByKey(
    (lambda x: (x, 1)),                        # createCombiner
    (lambda x, y: (x[0] + y, x[1] + 1)),       # mergeValue
    (lambda x, y: (x[0] + y[0], x[1] + y[1]))  # mergeCombiners
)

def extraccion(pair):
    clave = pair[0]
    valor = pair[1]
    tag = clave[0]
    issolved = clave[1]
    n_times = valor[0]
    n_questions = valor[1]
    if not issolved:
        return tag, (0, 'NaN', n_questions, n_times / n_questions)
    else:
        return tag, (n_questions, n_times / n_questions, 0, 'NaN')

estadisticasRDD = reducidoTagRDD.map(extraccion)

def reduccion(a, b):
    if a[1] == 'NaN':
        return b[0], b[1], a[2], a[3]
    else:
        return a[0], a[1], b[2], b[3]

RBK_RDD = estadisticasRDD.reduceByKey(reduccion)

RBK_RDD.sortBy(lambda a: a[1],False).collect()

Out[ 1 ]: [(u'Java', (3, 2, 1, 0)),
 (u'PHP', (2, 1, 0, 'NaN')),
 (u'Python', (1, 3, 1, 2)),
 (u'C#', (0, 'NaN', 2, 3))]